<center>
<h4>Diplomatura en CDAAyA 2019 - FaMAF - UNC</h4>
<h1>Predicción del Nivel de Pobreza de Hogares en Costa Rica</h1>
<h3>Exploración y Curación</h3>
</center>
</left>
<h4>Julieta Bergamasco</h4>
</left>

### Introducción

En la siguiente notebook, se presentará la consigna a seguir para el segundo práctico de la materia Exploración y Curación. El objetivo consiste en identificar e implementar los pasos necesarios para la limpieza de la base de datos de pobreza en hogares de Costa Rica, así como también analizar cruces de datos con mayor profundidad y validando el sentido lógico. Para ello, comenzaremos con las importaciones pertinentes.

### Importaciones

In [1]:
# Importación de las librerías necesarias
import numpy as np
import pandas as pd
# Puede que nos sirvan también
import matplotlib as mpl
mpl.get_cachedir()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 150)
pd.set_option('max_colwidth', 151)

## Consigna para Curación y Exploración del Dataset

### I. Rutina de Curación

Inicialmente, con el objetivo de preparar los datos que alimentarán futuros modelos de aprendizaje automático (ML), se propone seguir la siguiente __[checklist](https://dimewiki.worldbank.org/wiki/Checklist:_Data_Cleaning)__ para la limpieza de los datos de nuestro proyecto. Esta _checklist_ es la misma que utilizaron en el primer práctico de la materia y nos será de utilidad como guía para curar el dataset. A modo de ayuda, **en esta notebook encontrarán una especie de template** que sigue la _checklist_ y que deberán ir completando.

**Cada decisión tomada deberá quedar registrada de manera explícita y clara.** Luego de pasar por todos los puntos de la _checklist_ propuesta, deberán almacenar en un nuevo archivo los datos resultantes. Adicionalmente, deben tener en cuenta que las acciones de limpieza que realicen sobre este dataset también deberán ser implementadas sobre el dataset de test (que, por el momento, no ha sido disponibilizado).

A los fines de realizar este práctico, se utilizará el dataset original, pero descartando todas aquellas columnas que se hayan calculado en base a features preexistentes, ya que éstas están relacionadas a decisiones que adoptaremos más adelante, como por ejemplo, sobre si es necesario crear nuevas features y si incluirlas o no. Recuerden que la ciencia de datos es un **proceso circular, continuo y no lineal**. Es decir, si los datos requieren de mayor procesamiento para satisfacer las necesidades de algoritmos de ML (cualesquiera de ellos), vamos a volver a la etapa inicial para, por ejemplo, crear nuevas features, tomar decisiones diferentes sobre valores faltantes o valores atípicos (outliers), descartar features, entre otras.

### II. Análisis en Profundidad del Contenido

Una vez aplicada la _Checklist_, lo que vamos a hacer es profundizar aún más el análisis y tomar decisiones que se consideren pertinentes, si es que no lo han hecho aún en el desarrollo del primer apartado. Por supuesto, se deberán registrar todas las decisiones que tomen al respecto.

Al finalizar con el práctico, las preguntas listadas a continuación deberán quedar respondidas, mientras que si ya lo hicieron durante el desarrollo de la ´checklist´, el objetivo es que se replanteen las decisiones tomadas al respecto:

1. Hay tres features que, por su descripción, deberían ser numéricas: `dependency`, `edjefe` y `edjefa`. Sin embargo, el tipo de dato de estos campos es 'Objeto'. Obtener los valores únicos, analizar por qué no son numéricos y decidir cómo se transformarán a numéricos, teniendo en cuenta su significado.

2. Comparar los campos `tamhog` y `hhsize`. ¿Qué información nos da cada una? ¿Qué deberíamos hacer al respecto? ¿Qué diferencia hay con el campo `r4t3`?

3. ¿Qué ocurre con las rentas (`v2a1`) outliers? ¿Son razonables? ¿Mantenemos o descartamos estos registros?

4. ¿Para qué casos de las variables TipoVivienda (`tipovivi`) no existe renta? ¿Es un si y solo si? Es decir, por ejemplo, si son propietarios, nunca hay valor en el campo renta, mientras que si no lo son, siempre hay valor. En caso de que esto no sea así, ¿es razonable? ¿Qué decisión tomarían al respecto?

5. ¿Qué decidieron respecto a los valores faltantes de la renta? Una **propuesta que podría considerarse interesante** es completar tales valores faltantes con el **costo de oportunidad** de los propietarios de ese hogar. Es decir, ese hogar está dejando de percibir una renta igual a $x$ por estar habitando esa vivienda. Por lo tanto, ese costo de oportunidad puede estimarse de acuerdo al valor de la renta de viviendas similares, en base a múltiples criterios, como por ejemplo: zona, región, tipo de construcción, acceso a servicios y todas las características que ustedes consideren pertinentes. Esto quedará a su criterio. ¿Cuál es el único feature que no deberíamos usar para obtener el costo de oportunidad y por qué?

6. ¿Qué datos se repiten para `idhogar` y cuáles no? ¿Tiene sentido?

7. Si contamos la cantidad de individuos entrevistados por hogar, ¿coincide con el tamaño del hogar? ¿Y coincide con la suma de las variables que indican cantidad de personas? ¿Qué se puede hacer en los casos en que no?

8. Considerando los campos `rooms` y `bedrooms`, debería haber al menos tantos ambientes como habitaciones o bien más. Comprobar si esto es así efectivamente. Si no lo fuera, ¿puede ser un error de carga? ¿Cómo lo solucionarían?

9. ¿Todos los `idhogar` tienen jefa o jefe de hogar? (`parentesco1` == 1) Si esto no ocurre, ¿qué harían con los hogares que no tienen?

Esta lista es extensa e intenta abarcar todas las posibles irregularidades en los datos, pero puede no ser exhaustiva. **Cualquier análisis adicional de consistencia que deseen agregar porque lo consideran pertinente, será bienvenido y valorado.**

### Entregables

El entregable de este práctico consiste en esta misma Notebook, pero con la _checklist_ realizada y el análisis de contenido completo, explicando las decisiones tomadas en cada etapa. Además, deberán elaborar un script que contenga una función para curar nuevos datos con la misma estructura. Finalmente, deberán actualizar la metadata.

# Resolución

## I. Rutina de Curación

### 1. Importación de Datos

#### 1.1. Verificación de Inexistencia de Problemas en la Importación

Para comenzar, importamos los datos que vamos a procesar:

In [3]:
# Cargamos el Dataset original en una variable
# url = 'https://github.com/jbergamasco/DiploDatos2019/raw/master/ProyectoPobrezaCostaRica/DatasetPobCR_Train.txt'
url_data = 'https://github.com/jbergamasco/DiploDatos2019/raw/master/ProyectoPobrezaCostaRica/DatasetPobCR_Train.csv'
_ds = pd.read_csv(url_data)
_ds.head()

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,r4h2,r4h3,r4m1,r4m2,r4m3,r4t1,r4t2,r4t3,tamhog,tamviv,escolari,rez_esc,hhsize,paredblolad,paredzocalo,paredpreb,pareddes,paredmad,paredzinc,paredfibras,paredother,pisomoscer,pisocemento,pisoother,pisonatur,pisonotiene,pisomadera,techozinc,techoentrepiso,techocane,techootro,cielorazo,abastaguadentro,abastaguafuera,abastaguano,public,planpri,noelec,coopele,sanitario1,sanitario2,sanitario3,sanitario5,sanitario6,energcocinar1,energcocinar2,energcocinar3,energcocinar4,elimbasu1,elimbasu2,elimbasu3,elimbasu4,elimbasu5,elimbasu6,epared1,epared2,epared3,etecho1,etecho2,etecho3,eviv1,eviv2,eviv3,dis,male,female,estadocivil1,estadocivil2,estadocivil3,estadocivil4,estadocivil5,estadocivil6,estadocivil7,parentesco1,parentesco2,parentesco3,parentesco4,parentesco5,parentesco6,parentesco7,parentesco8,parentesco9,parentesco10,parentesco11,parentesco12,idhogar,hogar_nin,hogar_adul,hogar_mayor,hogar_total,dependency,edjefe,edjefa,meaneduc,instlevel1,instlevel2,instlevel3,instlevel4,instlevel5,instlevel6,instlevel7,instlevel8,instlevel9,bedrooms,overcrowding,tipovivi1,tipovivi2,tipovivi3,tipovivi4,tipovivi5,computer,television,mobilephone,qmobilephone,lugar1,lugar2,lugar3,lugar4,lugar5,lugar6,area1,area2,age,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
0,ID_279628684,190000.0,0,3,0,1,1,0,NaN,0,1,1,0,0,0,0,1,1,1,1,10,NaN,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,21eb7fcc1,0,1,0,1,no,10,no,10.0,0,0,0,1,0,0,0,0,0,1,1.000000,0,0,1,0,0,0,0,1,1,1,0,0,0,0,0,1,0,43,100,1849,1,100,0,1.000000,0.0,100.0,1849,4
1,ID_f29eb3ddd,135000.0,0,4,0,1,1,1,1.0,0,1,1,0,0,0,0,1,1,1,1,12,NaN,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0e5d7a658,0,1,1,1,8,12,no,12.0,0,0,0,0,0,0,0,1,0,1,1.000000,0,0,1,0,0,0,0,1,1,1,0,0,0,0,0,1,0,67,144,4489,1,144,0,1.000000,64.0,144.0,4489,4
2,ID_68de51c94,NaN,0,8,0,1,1,0,NaN,0,0,0,0,1,1,0,1,1,1,1,11,NaN,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,2c7317ea8,0,1,1,1,8,no,11,11.0,0,0,0,0,1,0,0,0,0,2,0.500000,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,92,121,8464,1,0,0,0.250000,64.0,121.0,8464,4
3,ID_d671db89c,180000.0,0,5,0,1,1,1,1.0,0,2,2,1,1,2,1,3,4,4,4,9,1.0,4,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,2b58d945f,2,2,0,4,yes,11,no,11.0,0,0,0,1,0,0,0,0,0,3,1.333333,0,0,1,0,0,0,0,1,3,1,0,0,0,0,0,1,0,17,81,289,16,121,4,1.777778,1.0,121.0,289,4
4,ID_d56d6f5f5,180000.0,0,5,0,1,1,1,1.0,0,2,2,1,1,2,1,3,4,4,4,11,NaN,4,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2b58d945f,2,2,0,4,yes,11,no,11.0,0,0,0,0,1,0,0,0,0,3,1.333333,0,0,1,0,0,0,0,1,3,1,0,0,0,0,0,1,0,37,121,1369,16,121,4,1.777778,1.0,121.0,1369,4


In [4]:
# Cargamos el Dataset Fields en una variable, para tener la información de cada campo
url_fields_info = 'https://github.com/jbergamasco/DiploDatos2019/raw/master/ProyectoPobrezaCostaRica/Dataset%20Fields.txt'
_data_fields = pd.read_csv(url_fields_info, sep='\t', encoding = 'ANSI')
_data_fields.head()

,Variable_name,Variable_description
0,v2a1,Monthly rent payment
1,hacdor,=1 Overcrowding by bedrooms
2,rooms,number of all rooms in the house
3,hacapo,=1 Overcrowding by rooms
4,v14a,=1 has bathroom in the household


Recuerden que la variable `Target` constituye nuestro objetivo de predicción. Es la etiqueta de los datos de acuerdo al nivel de pobreza del hogar que habitan, según la siguiente escala o clases:

1 = pobreza extrema<br>
2 = pobreza moderada<br>
3 = hogares vulnerables<br>
4 = hogares no vulnerables<br>

In [5]:
# Eliminamos columnas que son features calculadas en base a otros features
_calc_feat = _ds.loc[:,'SQBescolari':'agesq'].columns
print('Columnas eliminadas: ', _calc_feat.values)
_ds.drop(columns = _calc_feat, inplace = True)

Columnas eliminadas:  ['SQBescolari' 'SQBage' 'SQBhogar_total' 'SQBedjefe' 'SQBhogar_nin'
 'SQBovercrowding' 'SQBdependency' 'SQBmeaned' 'agesq']


Tomamos una muestra aleatoria para ver valores más dispersos.

In [6]:
# Fijar seed para asegurar reproducibilidad
np.random.seed(0)
_ds.sample(20)

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,r4h2,r4h3,r4m1,r4m2,r4m3,r4t1,r4t2,r4t3,tamhog,tamviv,escolari,rez_esc,hhsize,paredblolad,paredzocalo,paredpreb,pareddes,paredmad,paredzinc,paredfibras,paredother,pisomoscer,pisocemento,pisoother,pisonatur,pisonotiene,pisomadera,techozinc,techoentrepiso,techocane,techootro,cielorazo,abastaguadentro,abastaguafuera,abastaguano,public,planpri,noelec,coopele,sanitario1,sanitario2,sanitario3,sanitario5,sanitario6,energcocinar1,energcocinar2,energcocinar3,energcocinar4,elimbasu1,elimbasu2,elimbasu3,elimbasu4,elimbasu5,elimbasu6,epared1,epared2,epared3,etecho1,etecho2,etecho3,eviv1,eviv2,eviv3,dis,male,female,estadocivil1,estadocivil2,estadocivil3,estadocivil4,estadocivil5,estadocivil6,estadocivil7,parentesco1,parentesco2,parentesco3,parentesco4,parentesco5,parentesco6,parentesco7,parentesco8,parentesco9,parentesco10,parentesco11,parentesco12,idhogar,hogar_nin,hogar_adul,hogar_mayor,hogar_total,dependency,edjefe,edjefa,meaneduc,instlevel1,instlevel2,instlevel3,instlevel4,instlevel5,instlevel6,instlevel7,instlevel8,instlevel9,bedrooms,overcrowding,tipovivi1,tipovivi2,tipovivi3,tipovivi4,tipovivi5,computer,television,mobilephone,qmobilephone,lugar1,lugar2,lugar3,lugar4,lugar5,lugar6,area1,area2,age,Target
4319,ID_c9fdb7266,NaN,0,6,0,1,1,0,NaN,0,2,2,0,2,2,0,4,4,4,4,11,NaN,4,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,c92ca11d1,1,3,0,4,.33333334,no,6,9.333333,0,0,0,0,1,0,0,0,0,3,1.333333,0,0,0,0,1,1,1,1,4,1,0,0,0,0,0,1,0,25,4
3883,ID_fd6cc8def,NaN,0,4,0,1,1,1,1.0,0,0,0,0,2,2,0,2,2,2,2,14,NaN,2,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,72524240f,0,2,0,2,no,no,14,14.000000,0,0,0,0,0,0,0,1,0,2,1.000000,1,0,0,0,0,0,0,1,2,1,0,0,0,0,0,1,0,57,4
3775,ID_85d5cd65c,180000.0,0,4,0,1,1,1,2.0,1,1,2,1,1,2,2,2,4,4,4,15,NaN,4,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,006031de3,2,2,0,4,yes,15,no,15.500000,0,0,0,0,0,0,0,1,0,2,2.000000,0,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,39,4
1519,ID_c230376ae,NaN,0,5,0,1,1,1,2.0,0,1,1,1,1,2,1,2,3,3,3,12,NaN,3,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,c0480739d,2,1,0,3,2,no,16,28.000000,0,0,0,0,0,0,1,0,0,3,1.000000,1,0,0,0,0,0,1,1,3,1,0,0,0,0,0,1,0,19,4
8810,ID_991b84866,NaN,0,4,0,1,1,0,NaN,1,1,2,1,1,2,2,2,4,4,4,5,0.0,4,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,a8dcd4364,2,2,0,4,yes,9,no,9.500000,0,1,0,0,0,0,0,0,0,2,2.000000,1,0,0,0,0,0,0,1,4,0,0,0,0,0,1,1,0,11,2
4398,ID_6846cbaac,NaN,0,6,0,1,1,0,NaN,0,2,2,0,2,2,0,4,4,4,4,6,NaN,4,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,3677a8765,0,4,3,4,3,no,6,4.500000,0,0,1,0,0,0,0,0,0,3,1.333333,1,0,0,0,0,0,1,1,3,1,0,0,0,0,0,1,0,33,4
6632,ID_af3dd1bd2,NaN,0,7,0,1,1,1,1.0,0,0,0,1,2,3,1,2,3,3,3,12,NaN,3,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,351a9fd58,1,2,1,3,2,no,6,9.000000,0,0,0,0,0,0,1,0,0,3,1.000000,1,0,0,0,0,0,1,1,2,0,0,1,0,0,0,1,0,51,3
5790,ID_b6889eed0,NaN,0,3,0,1,1,0,NaN,0,1,1,0,2,2,0,3,3,3,3,11,NaN,3,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,ef6b27244,1,2,0,3,.5,no,8,15.000000,0,0,0,0,1,0,0,0,0,1,3.000000,1,0,0,0,0,0,0,1,3,0,1,0,0,0,0,1,0,58,1
3978,ID_22886534c,NaN,0,4,0,1,1,0,NaN,1,3,4,1,1,2,2,4,6,6,6,7,NaN,6,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0

Veamos los tipos de datos que tenemos

In [7]:
_ds.dtypes

Id                  object
v2a1               float64
hacdor               int64
rooms                int64
hacapo               int64
v14a                 int64
refrig               int64
v18q                 int64
v18q1              float64
r4h1                 int64
r4h2                 int64
r4h3                 int64
r4m1                 int64
r4m2                 int64
r4m3                 int64
r4t1                 int64
r4t2                 int64
r4t3                 int64
tamhog               int64
tamviv               int64
escolari             int64
rez_esc            float64
hhsize               int64
paredblolad          int64
paredzocalo          int64
paredpreb            int64
pareddes             int64
paredmad             int64
paredzinc            int64
paredfibras          int64
paredother           int64
pisomoscer           int64
pisocemento          int64
pisoother            int64
pisonatur            int64
pisonotiene          int64
pisomadera           int64
t

<div class="alert alert-block alert-info">
El dataset ya está **listo para trabajar!**
</div>

### 1.2. Asegurar la Existencia de IDs o Claves Únicas

El siguiente paso implica chequear que no existen datos duplicados y que las claves, si existen, son únicas.

### 1.3. Despersonalizar Datos y Guardarlos en un Nuevo Archivo

En este caso, no es necesario despersonalizar ningún dato, dado que los mismos ya se encuentran despersonalizados.

### 1.4. Nunca Modificar los Datos Crudos u Originales

Al finalizar la limpieza, deberán guardar el dataset resultante, para asegurarse de no modificar los datos originales.

## 2. Pasos de Limpieza Necesarios

### 2.1. Etiquetas de Variables/Columnas y Problemas de Codificación/Encoding

Antes que nada, verificar el encoding de la fuente de datos, leyendo en crudo los primeros 100000 caracteres, por ejemplo:

In [8]:
import chardet

In [9]:
import requests
rawdata = requests.get(url_data)
result = chardet.detect(rawdata.content[:100000])
result

{'confidence': 1.0, 'encoding': 'UTF-8-SIG', 'language': ''}

Analizar los nombres de columnas, utilizando por ejemplo `_ds.columns.str.extract(r'^(\w+)$')`.

In [10]:
_ds.columns.values

array(['Id', 'v2a1', 'hacdor', 'rooms', 'hacapo', 'v14a', 'refrig',
       'v18q', 'v18q1', 'r4h1', 'r4h2', 'r4h3', 'r4m1', 'r4m2', 'r4m3',
       'r4t1', 'r4t2', 'r4t3', 'tamhog', 'tamviv', 'escolari', 'rez_esc',
       'hhsize', 'paredblolad', 'paredzocalo', 'paredpreb', 'pareddes',
       'paredmad', 'paredzinc', 'paredfibras', 'paredother', 'pisomoscer',
       'pisocemento', 'pisoother', 'pisonatur', 'pisonotiene',
       'pisomadera', 'techozinc', 'techoentrepiso', 'techocane',
       'techootro', 'cielorazo', 'abastaguadentro', 'abastaguafuera',
       'abastaguano', 'public', 'planpri', 'noelec', 'coopele',
       'sanitario1', 'sanitario2', 'sanitario3', 'sanitario5',
       'sanitario6', 'energcocinar1', 'energcocinar2', 'energcocinar3',
       'energcocinar4', 'elimbasu1', 'elimbasu2', 'elimbasu3',
       'elimbasu4', 'elimbasu5', 'elimbasu6', 'epared1', 'epared2',
       'epared3', 'etecho1', 'etecho2', 'etecho3', 'eviv1', 'eviv2',
       'eviv3', 'dis', 'male', 'female', '

### 2.2. Tratamiento de Valores Faltantes

Para analizar los valores faltantes, primero deberán saber cuántos existen por campo y cuánto representan del total:

In [11]:
valores_faltantes = pd.DataFrame([_ds.isnull().sum(),
                                   _ds.isnull().sum()/len(_ds)]).transpose().rename(
    columns = {0:'Cantidad_NaN',1:'Porcentaje_Nan_s_Total'})

valores_faltantes.loc[valores_faltantes['Cantidad_NaN']>0].style.format({'Porcentaje_Nan_s_Total':"{:.2%}"})

,Cantidad_NaN,Porcentaje_Nan_s_Total
v2a1,6860,71.78%
v18q1,7342,76.82%
rez_esc,7928,82.95%
meaneduc,5,0.05%


### 2.3. Codificación de Variables Categóricas

Aplica?

### 2.4. No Cambiar los Nombres de las Variables de la Fuente de Origen

### 2.5. Verificación de Consistencia de Datos

Este es el paso más analítico, en donde se deben aplicar reglas de integridad.

### 2.6. Identificar y Documentar Valores Atípicos/Outliers

Calcular estadísticos.

### 2.7. Evaluar Cómo Comprimir los Datos Para su Almacenamiento Más Eficiente

### 2.8. Guardar el Set de Datos con un Nombre Informativo

## 3. Pasos de Limpieza Deseables

### 3.1. Ordenar Variables/Columnas

### 3.2. Quitar Variables/Columnas Irrelevantes

Tenemos alguna en este caso?

### 3.3. Renombrar Variables de Grillas

Esto se puede hacer de manera transparente a través de un diccionario.
Sugiero darles nombres más informativos a algunas columnas, como por ejemplo la de Renta Mensual y las de lugares. Como ayuda, sigue el diccionario de las columnas `lugar` con su nuevo nombre haciendo referencia directa a la región:

In [12]:
# lugar1 =1 region Central
# lugar2 =1 region Chorotega
# lugar3 =1 region Pacífico central
# lugar4 =1 region Brunca
# lugar5 =1 region Huetar Atlántica
# lugar6 =1 region Huetar Norte

_col_dict = {'lugar1': 'Central',
             'lugar2': 'Chorotega',
             'lugar3': 'Pacífico Central',
             'lugar4': 'Brunca',
             'lugar5': 'Huetar Atlántica',
             'lugar6': 'Huetar Norte'}

### 3.4. Categorizar Variables que Contengan “Otros”

### 3.5. Agregar Metadata a los Datos

Cuando y como fueron obtenidos, limpieza realizada, decisiones implementadas, asunciones, etc.

## II. Análisis en Profundidad del Contenido

### 1. Features Tipo Objetos

Primero nos quedamos con las features cuyo tipo es 'Objeto'.

In [13]:
# Observamos los campos que tienen tipo 'Objeto'
_ds.dtypes[_ds.dtypes == 'object']

Id            object
idhogar       object
dependency    object
edjefe        object
edjefa        object
dtype: object

`Id` e `idhogar`, qué son?

In [14]:
# Observamos los valores posibles del campo "Dependency"
_dep_arr = _ds['dependency'].unique()
_dep_arr.sort()
_dep_arr[::-1]

array(['yes', 'no', '8', '6', '5', '4', '3.5', '3', '2.5', '2.25', '2',
       '1.75', '1.6666666', '1.5', '1.3333334', '1.25', '1.2',
       '.83333331', '.80000001', '.75', '.71428573', '.66666669',
       '.60000002', '.5', '.40000001', '.33333334', '.2857143', '.25',
       '.22222222', '.2', '.16666667'], dtype=object)

In [15]:
# Observamos los valores posibles del campo "Edjefe"
_arr = _ds['edjefe'].unique()
_arr.sort()
_arr[::-1]

array(['yes', 'no', '9', '8', '7', '6', '5', '4', '3', '21', '20', '2',
       '19', '18', '17', '16', '15', '14', '13', '12', '11', '10'],
      dtype=object)

In [16]:
# Analizamos el significado del campo "Edjefe"
_data_fields[_data_fields['Variable_name'] == 'edjefe']

,Variable_name,Variable_description
100,edjefe,"years of education of male head of household, based on the interaction of escolari (years of education), head of household and gender, yes=1 and no=0"


In [17]:
# Observamos los valores posibles del campo "Edjefa"
_arr = _ds['edjefa'].unique()
_arr.sort()
_arr[::-1]

array(['yes', 'no', '9', '8', '7', '6', '5', '4', '3', '21', '20', '2',
       '19', '18', '17', '16', '15', '14', '13', '12', '11', '10'],
      dtype=object)

### 2. Campos `tamhog` y `hhsize`

In [18]:
# Tenemos dos campos iguales!!
_ds['tamhog'].equals(_ds['hhsize'])

True

### 3. Renta Mensual: Outliers

### 4. Renta Mensual y Tipo de Vivienda

### 5. Renta Mensual: Valores Faltantes

### 6. Datos Únicos por `idhogar`

### 7. Individuos por Hogar

### 8. `rooms` y `bedrooms`

### 9. Jef@s de Hogar por `idhogar`

### 10. Adicional